# Code Notebook 1: Introduction and Missing Values #

This notebook introduces the purpose of the project and includes initial data cleaning.  This involves indentifying all types of missing values in numeric features and ensuring that all features have appropriate data types.  Other possible problems, such as outliers, are addressed in later notebooks.  It is also more convenient to handle missing values in categorical data later, while numerically encoding such data.

## I. Problem Statement ##

The goal of this project is to use the Ames Housing Dataset, found [here](http://jse.amstat.org/v19n3/decock/DataDocumentation.txt), to assist stakeholders with an interest in increasing home value.  Linear regression will be used to design a model that can accept important features from the data set and make predictions of sale price based on those features.  In addition to providing a method for estimating home value given specific features, this approach will provide measures of importance for included features.  This will allow me to provide interested parties with information about the most important types of features to focus on if they wish to increase the value of a home.

To test model effectiveness, I will compare it with the naive model of estimating all home values to be equal to the mean of all sale prices in the data set.  We should expect significant improvement over this model.  To measure this, I will use the R2 score, which will tell me how much of the price variability my model is able to account for relative to the baseline model.  I will also use error, which gives a direct comparison of average error in estimates made with the naive model vs. my regression model.  Root mean square tends to punish large outiers severely, making it ideal for a situation in which it's worse to be wrong by a lot than to be wrong by a little.  Root mean square will provide a measure of prediction accuracy expressed in dollars, ideal for making comparisons between the baseline model and regression model.

In [1]:
import numpy as np
import pandas as pd

import cleantools as ct

In [2]:
# Importing all data.  To be safe, I'm not having Pandas automatically identify NaN values.  I'll find and interpret them myself.
train = pd.read_csv('../datasets/train.csv', keep_default_na=False)
test = pd.read_csv('../datasets/test.csv', keep_default_na=False)

## II. Examining NaN/Missing Values ##

In [3]:
# Using custom function to convert all column names to snake case.
train = ct.df_cols_to_snake_case(train)
test = ct.df_cols_to_snake_case(test)

In [4]:
# Using custom function to store data type and number of NaNs in each column in a DataFrame.
train_info = ct.info_frame(train)

In [5]:
# As expected, there are no NaN values in my data.
train_info['num_nans'].value_counts()

0    81
Name: num_nans, dtype: int64

In [6]:
# There are only two data types: int and object.
train_info['type'].value_counts()

object    53
int64     28
Name: type, dtype: int64

In [7]:
# This variable holds the names of all the columns of type int, of which there are 28.
int_cols = [col for col in train.columns if train[col].dtype != object]
print(len(int_cols))

28


In [8]:
# This variable holds the names of all the columns of type object, of which there are 53.
object_cols = [col for col in train.columns if train[col].dtype == object]
print(len(object_cols))

53


In [9]:
train[object_cols[:15]].head(3)

,ms_zoning,lot_frontage,street,alley,lot_shape,land_contour,utilities,lot_config,land_slope,neighborhood,condition_1,condition_2,bldg_type,house_style,roof_style
0,RL,,Pave,NA,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,Gable
1,RL,43,Pave,NA,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,Gable
2,RL,68,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Gable


In [10]:
train[object_cols[15:28]].head(3)

,roof_matl,exterior_1st,exterior_2nd,mas_vnr_type,mas_vnr_area,exter_qual,exter_cond,foundation,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1
0,CompShg,HdBoard,Plywood,BrkFace,289,Gd,TA,CBlock,TA,TA,No,GLQ,533
1,CompShg,VinylSd,VinylSd,BrkFace,132,Gd,TA,PConc,Gd,TA,No,GLQ,637
2,CompShg,VinylSd,VinylSd,None,0,TA,Gd,CBlock,TA,TA,No,GLQ,731


In [11]:
train[object_cols[28:40]].head(3)

,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,heating,heating_qc,central_air,electrical,bsmt_full_bath,bsmt_half_bath,kitchen_qual,functional
0,Unf,0,192,725,GasA,Ex,Y,SBrkr,0,0,Gd,Typ
1,Unf,0,276,913,GasA,Ex,Y,SBrkr,1,0,Gd,Typ
2,Unf,0,326,1057,GasA,TA,Y,SBrkr,1,0,Gd,Typ


In [12]:
train[object_cols[40:]].head(3)

,fireplace_qu,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond,paved_drive,pool_qc,fence,misc_feature,sale_type
0,NA,Attchd,1976,RFn,2,475,TA,TA,Y,NA,NA,NA,WD
1,TA,Attchd,1997,RFn,2,559,TA,TA,Y,NA,NA,NA,WD
2,NA,Detchd,1953,Unf,1,246,TA,TA,Y,NA,NA,NA,WD


I have imported my data using the option not to automatically identify any NaN values.  This resulted in only int and object data types.  Above, I have displayed the first few rows of each column that has data type object.  We can see that some of these columns should be numeric, so let's see what's going on.

Below, I am checking any columns with type object that look like they may actually be numeric data.  I'm doing this by identifying any data in those columns that can't be cast as numeric.  I then use .value_counts() to learn more about the number of such entries and whether 0 or something similar is also present.  The absence of 0 may mean that a missing value means 0.

In [13]:
# Using my custom function to check columns with type object that might actually be numeric data.
ct.find_non_numeric(train['lot_frontage'])

['']

In [14]:
train['lot_frontage'].value_counts().head()

      330
60    179
70     96
80     94
50     90
Name: lot_frontage, dtype: int64

All the non-numeric values in 'lot_frontage' are empty strings. 0 is not present.

In [15]:
ct.find_non_numeric(train['alley'])

['NA', 'Pave', 'Grvl']

In [16]:
train['alley'].value_counts()

NA      1911
Grvl      85
Pave      55
Name: alley, dtype: int64

'alley' is categorical, and those NA are numerous enough to mean something (probably that there is no alley.)

In [17]:
ct.find_non_numeric(train['mas_vnr_area'])

['']

In [18]:
train['mas_vnr_area'].value_counts().head()

0      1216
         22
120      11
200      10
176      10
Name: mas_vnr_area, dtype: int64

All the non-numeric values in 'mas_vnr_area' are empty strings. 0 is present and much more frequent than empty.

In [19]:
ct.find_non_numeric(train['bsmtfin_sf_1'])

['']

In [20]:
train['bsmtfin_sf_1'].value_counts().head()

0      657
24      17
16      10
20       7
384      7
Name: bsmtfin_sf_1, dtype: int64

All the non-numeric values in 'bsmtfin_sf_1' are empty strings.  0 is present and much more frequent than empty.

In [21]:
ct.find_non_numeric(train['bsmtfin_sf_2'])

['']

In [22]:
train['bsmtfin_sf_2'].value_counts().head()

0      1803
294       5
162       3
72        3
180       3
Name: bsmtfin_sf_2, dtype: int64

All the non-numeric values in 'bsmtfin_sf_2' are empty strings. 0 is present and much more frequent than empty.

In [23]:
ct.find_non_numeric(train['bsmt_unf_sf'])

['']

In [24]:
train['bsmt_unf_sf'].value_counts().head()

0      165
384     14
672     11
728     10
816      9
Name: bsmt_unf_sf, dtype: int64

All the non-numeric values in 'bsmt_unf_sf' are empty strings.  0 is present and much for frequent than empty.

In [25]:
ct.find_non_numeric(train['total_bsmt_sf'])

['']

In [26]:
train['total_bsmt_sf'].value_counts().head()

0       54
864     51
1040    20
672     20
816     17
Name: total_bsmt_sf, dtype: int64

All the non-numeric values in 'total_bsmt_sf' are empty strings.  0 is present and much for frequent than empty.

In [27]:
ct.find_non_numeric(train['bsmt_full_bath'])

['']

In [28]:
train['bsmt_full_bath'].value_counts().head()

0    1200
1     824
2      23
        2
3       2
Name: bsmt_full_bath, dtype: int64

All the non-numeric values in 'bsmt_full_bath' are empty strings.  0 is present and much for frequent than empty.

In [29]:
ct.find_non_numeric(train['bsmt_half_bath'])

['']

In [30]:
train['bsmt_half_bath'].value_counts().head()

0    1923
1     122
2       4
        2
Name: bsmt_half_bath, dtype: int64

All the non-numeric values in 'bsmt_half_bath' are empty strings.  0 is present and much for frequent than empty.

In [31]:
ct.find_non_numeric(train['garage_yr_blt'])

['']

In [32]:
train['garage_yr_blt'].value_counts().head()

        114
2005    105
2007     84
2006     80
2004     72
Name: garage_yr_blt, dtype: int64

All the non-numeric values in 'garage_yr_blt' are empty strings. In this case, empty could refer to either unknown or the lack of a garage.

In [33]:
ct.find_non_numeric(train['garage_cars'])

['']

In [34]:
train['garage_cars'].value_counts().head()

2    1136
1     524
3     263
0     113
4      13
Name: garage_cars, dtype: int64

All the non-numeric values in 'garage_cars' are empty strings.  0 is present and more frequent than empty.

In [35]:
ct.find_non_numeric(train['garage_area'])

['']

In [36]:
train['garage_area'].value_counts().head()

0      113
440     70
576     69
484     52
528     46
Name: garage_area, dtype: int64

All the non-numeric values in 'garage_area' are empty strings. 0 is present and much more frequent than empty.

In [37]:
ct.find_non_numeric(train['pool_qc'])

['NA', 'Fa', 'Gd', 'Ex', 'TA']

In [38]:
train['pool_qc'].value_counts()

NA    2042
Gd       4
TA       2
Fa       2
Ex       1
Name: pool_qc, dtype: int64

'pool_qc' is categorical, and most of its entries are NA.

In [39]:
ct.find_non_numeric(train['fence'])

['NA', 'MnPrv', 'GdPrv', 'GdWo', 'MnWw']

In [40]:
train['fence'].value_counts()

NA       1651
MnPrv     227
GdPrv      83
GdWo       80
MnWw       10
Name: fence, dtype: int64

'fence' is categorical, and most of its values are NA.

In [41]:
ct.find_non_numeric(train['misc_feature'])

['NA', 'Shed', 'TenC', 'Gar2', 'Othr', 'Elev']

In [42]:
train['misc_feature'].value_counts()

NA      1986
Shed      56
Gar2       4
Othr       3
TenC       1
Elev       1
Name: misc_feature, dtype: int64

'misc_feature' is categorical, and most if its values are NA.

Many of these columns should probably be coded as numeric with NaN values.  I will decide how to handle the NaN values during feature engineering.  Before I do this, though, it's worth considering for each column why the values may be missing, and whether there is a better option than coding them as NaN.

- 'lot_frontage' --> Linear feet of street connected to property.
    - It's possible that there is no street connected to the properties with missing values.
    - Ames is a small town surrounded by farmland.
    - 'lot_config' also has to do with frontage and may help shed light on this.
- 'mas_vnr_area' --> Masonry veneer area in square feet.
    - Empty may mean no veneer, but 0 is also present as a value, so I can't assume that.
    - This will be worth cross-referencing with 'mas_vnr_type'
- 'bsmtfin_sf_1' --> Type 1 finished square fee.
    - Empty could mean no finishing, but 0 is also present.  There are very few houses with this value empty.
    - It may help to cross-reference with other features involving basements.
- 'bsmtfin_sf_2' --> Type 2 finished square fee.
    - Similar to 'bsmtfin_sf_1'
- 'bsmt_unf_sf' --> Unfinished square feet of basement area.
- 'total_bsmt_sf' --> Total square feet of basement area.
- 'bsmt_full_bath' --> Basement full bathrooms.
- 'bsmt_half_bath' --> Basement half bathrooms.
- 'garage_yr_blt' --> Year garage was built.
    - Cross reference with other garage variables.
- 'garage_cars' --> Size of garage in car capacity.
- 'garage_area' --> Size of garage in square feet.

#### *'lot_config' and 'lot_frontage'* ####

In [43]:
# Show lot configuration for all listings with empty value for lot frontage.
train['lot_config'][train['lot_frontage'] == ''].value_counts()

Inside     190
Corner      67
CulDSac     61
FR2         11
FR3          1
Name: lot_config, dtype: int64

In [44]:
train['lot_config'].value_counts()

Inside     1503
Corner      348
CulDSac     131
FR2          60
FR3           9
Name: lot_config, dtype: int64

Some of the listings with no value listed for lot frontage are also listed as having frontage on multiple sides.  Further investigation reveals that the only lot configurations in the data set are "inside," "corner," "cul-de-sec," "frontage on 2 sides," and "frontage on 3 sides."  This confirms that all listings do have at least some kind of frontage ([source](https://journal.firsttuesday.us/type-of-lots/70394/)).  For every lot configuration, there are some with a value for 'lot_frontage' and some without.  Therefore, these missing values cannot be easily accounted for.  They will be filled with NaN until feature selection.

#### *'mas_vnr_type' and 'mas_vnr_area'* ####

In [45]:
train['mas_vnr_type'].value_counts()

None       1218
BrkFace     630
Stone       168
             22
BrkCmn       13
Name: mas_vnr_type, dtype: int64

In [46]:
# Show masonry veneer type for all listings with empty value for masonry veneer area.
train['mas_vnr_type'][train['mas_vnr_area'] == ''].value_counts()

    22
Name: mas_vnr_type, dtype: int64

In [47]:
# Show masonry veneer type for all listings with 0 value for masonry veneer area.
train['mas_vnr_type'][train['mas_vnr_area'] == '0'].value_counts()

None       1213
BrkFace       2
Stone         1
Name: mas_vnr_type, dtype: int64

In [48]:
train['mas_vnr_area'][train['mas_vnr_type'] == 'None'].value_counts()

0      1213
1         3
288       1
344       1
Name: mas_vnr_area, dtype: int64

The missing values for masonry veneer type and masonry veneer area correspond exactly.  It's reasonable to guess that they should be None or 0.  This will be explored during feature selection.

There are a few nonsense values where the veneer area is 0 but type is other than None, or vice versa.  However, there aren't many such instances, and I can't do much about this, as it would be difficult to spot and account for such instances on unseen data.

#### *'bsmt' Features* ####

In [49]:
# Create a list of the basement features.
bsmt_features = [col for col in train.columns if 'bsmt' in col]

In [50]:
# Show only the rows for which at least one basement feature is ''.
train[bsmt_features][train[bsmt_features].isin(['']).any(axis=1)]

,bsmt_qual,bsmt_cond,bsmt_exposure,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
616,NA,NA,NA,NA,0,NA,0,0,0,,
1147,Gd,TA,No,GLQ,1124,,479,1603,3206,1,0
1327,,,,,,,,,,,
1456,Gd,TA,,Unf,0,Unf,0,725,725,0,0
1547,Gd,TA,,Unf,0,Unf,0,1595,1595,0,0
1997,Gd,TA,,Unf,0,Unf,0,936,936,0,0


It is highly likely that all the missing numeric entries should be 0.  These will be filled with 0 at the end of this section.
- 616 has no basement, and the basement bath features were left empty.
- 1327 likely also has no basement, as none of those fields are filled.

The categorical variables will be revisited during feature engineering.

#### *'garage' Features* ####

In [51]:
# Create a list of the garage features.
garage_features = [col for col in train.columns if 'garage' in col]
garage_features

['garage_type',
 'garage_yr_blt',
 'garage_finish',
 'garage_cars',
 'garage_area',
 'garage_qual',
 'garage_cond']

In [52]:
# Show only the rows for which at least one garage feature is ''.  Showing only the first 5.
train[garage_features][train[garage_features].isin(['']).any(axis=1)].head()

,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond
28,NA,,NA,0,0,NA,NA
53,NA,,NA,0,0,NA,NA
65,NA,,NA,0,0,NA,NA
79,NA,,NA,0,0,NA,NA
101,NA,,NA,0,0,NA,NA


The problem is that year build is a numeric column, so the only reasonable option for it when there is no garage is to leave it blank.  Let's take year built out of the list temporarily and see what's left.

In [53]:
# How many empty values are in the garage features other than year built?
garage_features.remove('garage_yr_blt')
print(train[garage_features][train[garage_features].isin(['']).any(axis=1)].head())
garage_features.append('garage_yr_blt')

     garage_type garage_finish garage_cars garage_area garage_qual garage_cond
1712      Detchd                                                              


The only problem left is 1712, which says it has a detached garage but has no further information about it.  The missing values most likely represent missing information about a real garage.  Since there is only one row like this, I will fill the numeric columns with the mean for all other garages with type 'Detchd'.

Because missing 'garage_yr_blt' values mostly indicate lack of a garage, those will be filled with NaN until further options are explored during feature selection.

## III. Numeric NaN Pipeline ##

In [54]:
def pipe_float_na(df):

    # Replace empty values with NaN and typecast column to float.
    df['lot_frontage'] = df['lot_frontage'].apply(ct.convert_to_float)
    df['mas_vnr_area'] = df['mas_vnr_area'].apply(ct.convert_to_float)
    df['garage_yr_blt'] = df['garage_yr_blt'].apply(ct.convert_to_float)

    # Select only the numeric basement features.
    bsmt_numeric_features = [col for col in bsmt_features if ('sf' in col) or ('bath' in col)]
    # Replace '' with 0.
    # I found out how to make a tuple with one element here:
    # https://stackoverflow.com/questions/12876177/how-to-create-a-tuple-with-only-one-element
    for col in bsmt_numeric_features:
        df[col] = df[col].apply(ct.convert_to_float, args=(0,))

    # Replace missing values in numeric garage columns with NaN and typecast to float.
    for col in ['garage_cars', 'garage_area']:
        df[col] = df[col].apply(ct.convert_to_float)
    # Fill the NaN values in the numeric garage colummns with the average over all columns with type 'Detchd'
    df['garage_cars'].fillna(df['garage_cars'][df['garage_type'] == 'Detchd'].mean(), inplace=True)
    df['garage_area'].fillna(df['garage_area'][df['garage_type'] == 'Detchd'].mean(), inplace=True)
    
    return df

In [55]:
train = pipe_float_na(train)
test = pipe_float_na(test)

In [56]:
train.to_csv('../datasets/train_1.csv', index=False, na_rep='null')
test.to_csv('../datasets/test_1.csv', index=False, na_rep='null')

## IV. Summary: Missing Values ##

NOTE: There are now 3 column types: 'object' means strings, 'int' means numeric with no NaN, 'float' means numeric possibly with some NaN.

- 'lot_frontage' has several NaN values that must either be dropped or replaced with reasonable guesses.
- 'mas_vnr_type' and 'mas_vnr_area' have 22 empty or NaN values that perfectly coincide.  They likely should be None or 0.  This can be experimented with during feature engineering.
    - 'mas_vnr_type' and 'mas_vnr_area' are contradictory for a very small number of entries.
- Numeric 'bsmt' colums had a very small number of missing values replaced with 0, which is likely the correct value.
- 'garage_yr_blt' NaN values mean there is no garage.
- 'garage_cars' and 'garage_area' each had a single missing value replaced with their average for the garage type 'Detchd.'

Some of the 'object' type columns may still have empty values that need to be replaced with NaN, but those will be better handled during feature engineering.